<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/%E0%B9%80%E0%B9%80%E0%B8%9A%E0%B9%88%E0%B8%87%E0%B8%A7%E0%B9%88%E0%B8%B2%E0%B9%83%E0%B8%84%E0%B8%A3%E0%B8%9E%E0%B8%B9%E0%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
!pip install attacut
!pip install pythainlp
!pip install epitran

In [194]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
import matplotlib as mpl
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [195]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อมูลFacebook 2 - Sheet1.csv')

In [196]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [197]:
comment

,ชื่อ,คำพูดโรค
0,Akekapong Bowling Singharas,วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะ...
1,Aliis Chattarika,Theboom Moungna
2,Alisa Chuasa,Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา
3,Amornrat Pollamuangdee,คลำเจอก้อนที่เต้านมค่ะ
4,Anothai Hassabut,คัดกรองเบื้องต้นด้วยการตรวจเต้านมด้วยตัวเองว่า...
...,...,...
276,แม่รุ้ง น้องเกล,ลูกสาวมีอาการปวดท้องแบบแน่นๆ เวียนหัว มีจุดจ้ำ...
277,แอนนี่ น้องข้าวโอ๊ต,Wanitchaya Puruean
278,แอม' มม มมมมมมม.,มีก้อนเนื้อที่เต้านม ไม่มีอาการเจ็บ แต่คลำๆแล้...
279,โบว์ คะ,จับนมแล้วเจอก้อน


In [226]:
def detect_person(comment):
    # คำที่ใช้ตรวจสอบว่ามีใครเป็นคนอยู่ในความคิดเห็น
    other = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา','แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย',
              'ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเฟน','พ่อผม','เเม่ผม','พ่อหนู','เเม่หนู','พ่อพม','เเม่พม']
    myself = ['ผมเป็น','ผมเอง','กระผม','พมเป็น','พมเอง','กระพม','หนูเอง','หนู','ดิฉัน','ตัวเอง']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in  other:
        if keyword in comment and "อาการ" not in comment :
              return 'เล่าประสบการณ์คนอื่น'
    for keyword in  myself:
        if keyword in comment:
            return 'เล่าประสบการณ์ตัวเอง'
    # หากไม่พบคำที่บ่งบอกถึงคน
    return 'ไม่ได้เล่าประสบการณ์'

In [227]:
detect_person(comment['คำพูดโรค'][7])

'ไม่ได้เล่าประสบการณ์'

In [228]:
def detect_gender_other(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา','พ่อผม','พ่อหนู','พ่อพม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment and "อาการ" not in comment :
              return "ชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "หญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [229]:
comment['คำพูดโรค'][9]

'ผมพูดตามความจริงที่เจอนะครับ มะเร็งปากมดลูกของแม่ผมไม่มีอาการที่บ่งบอกได้ชัดว่ากำลังเป็นมะเร็ง มามีอาการอีกทีก็ระยะสุดท้ายและแล้วมันก็พรากแม่ผมไป แนะนำให้ทุกคนตรวจสุขภาพทุกปีนะครับ'

In [230]:
detect_person(comment['คำพูดโรค'][9])

'ไม่ได้เล่าประสบการณ์'

In [231]:
detect_gender_other(comment['คำพูดโรค'][9])

'หญิง'

In [201]:
def detect_gender_self(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['ผมเป็น','ครับ','ผมเอง','คับ','กระผม','พมเป็น','พมเอง','กระพม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['ค่ะ','คะ','หนูเอง','หนูเป็น','ดิฉัน']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment:
            return "ชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "หญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [203]:
use = comment
k1=[]
k2=[]
for i in comment['คำพูดโรค']:
  k1.append(detect_person(i))
  if detect_person(i) == 'เล่าประสบการณ์คนอื่น':
    k2.append(detect_gender_other(i))
  elif detect_person(i) == 'เล่าประสบการณ์ตัวเอง':
    k2.append(detect_gender_self(i))
  elif detect_person(i) == 'ไม่ได้เล่าประสบการณ์':
    k2.append(detect_gender_self(i))
use['ใครเล่า'] = k1
use['เพศ'] = k2

In [204]:
use

,ชื่อ,คำพูดโรค,ใครเล่า,เพศ
0,Akekapong Bowling Singharas,วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะ...,ไม่ได้เล่าประสบการณ์,ชาย
1,Aliis Chattarika,Theboom Moungna,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ
2,Alisa Chuasa,Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ
3,Amornrat Pollamuangdee,คลำเจอก้อนที่เต้านมค่ะ,ไม่ได้เล่าประสบการณ์,หญิง
4,Anothai Hassabut,คัดกรองเบื้องต้นด้วยการตรวจเต้านมด้วยตัวเองว่า...,เล่าประสบการณ์ตัวเอง,ไม่ระบุเพศ
...,...,...,...,...
276,แม่รุ้ง น้องเกล,ลูกสาวมีอาการปวดท้องแบบแน่นๆ เวียนหัว มีจุดจ้ำ...,เล่าประสบการณ์คนอื่น,หญิง
277,แอนนี่ น้องข้าวโอ๊ต,Wanitchaya Puruean,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ
278,แอม' มม มมมมมมม.,มีก้อนเนื้อที่เต้านม ไม่มีอาการเจ็บ แต่คลำๆแล้...,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ
279,โบว์ คะ,จับนมแล้วเจอก้อน,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ


In [215]:
# นิยามฟังก์ชันสำหรับค้นหาคำว่า "อา" แต่ไม่เอาคำที่มีอาการ
def ค้นหา_อา(ข้อความ):
    คำทั้งหมด = ข้อความ.split()
    คำที่ค้นพบ = []
    for คำ in คำทั้งหมด:
        if "อา" in คำ and "อาการ" not in คำ:
            คำที่ค้นพบ.append(คำ)
    return คำที่ค้นพบ

# ข้อความที่ต้องการค้นหา
ข้อความ = "อากาศในวันนี้ดีมาก อาหารเช้าที่ร้านนั้นอร่อยมาก อาชีพของเขาเป็นหมอ"

# ค้นหาคำ "อา" แต่ไม่เอาคำที่มี "อาการ"
ผลลัพธ์ = ค้นหา_อา(ข้อความ)
print("คำที่พบ:", ผลลัพธ์)

คำที่พบ: ['อากาศในวันนี้ดีมาก', 'อาหารเช้าที่ร้านนั้นอร่อยมาก', 'อาชีพของเขาเป็นหมอ']
